In [1]:
import os
import tarfile
import urllib.request
from pathlib import Path

DOWNLOAD_ROOT = "http://spamassassin.apache.org/old/publiccorpus/"
HAM_URL = DOWNLOAD_ROOT + "20030228_easy_ham.tar.bz2"
SPAM_URL = DOWNLOAD_ROOT + "20030228_spam.tar.bz2"
DATA_PATH = Path('datasets')
SPAM_PATH = DATA_PATH / 'spam'

In [2]:
if not DATA_PATH.is_dir():
    DATA_PATH.mkdir()
if not SPAM_PATH.is_dir():
    SPAM_PATH.mkdir()

In [3]:
def fetch_spam_data(spam_url=SPAM_URL, spam_path=SPAM_PATH):
    for filename,url in [('ham.tar.bz2',HAM_URL),('spam.tar.bz2',SPAM_URL)]:
        path=SPAM_PATH / filename
        if not path.is_file():
            with urllib.request.urlopen(url) as response:
                downloadfile = response.read()
                with open(path,'wb') as f:
                    f.write(downloadfile)
        with tarfile.open(path) as tf:
            tf.extractall(path=SPAM_PATH)

In [4]:
fetch_spam_data()

In [2]:
HAM_DIR = SPAM_PATH / 'easy_ham'
SPAM_DIR = SPAM_PATH / 'spam'
ham_filenames = [name for name in os.listdir(HAM_DIR) if len(name)>10]
spam_filename = [name for name in os.listdir(SPAM_DIR) if len(name)>10]

In [3]:
len(ham_filenames), len(spam_filename)

(2500, 500)

In [4]:
import email
import email.policy

In [5]:
def load_email(is_spam, filename, spam_path=SPAM_PATH):
    directory = 'spam' if is_spam else 'easy_ham'
    with open(spam_path/directory/filename, 'rb') as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

In [6]:
ham_emails = [load_email(False,name) for name in ham_filenames]
spam_emails = [load_email(True,name) for name in spam_filename]

In [10]:
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [7]:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        return 'multipart({})'.format(', '.join([
            get_email_structure(sub_email) 
            for sub_email in payload]))
    else:
        return email.get_content_type()

In [8]:
from collections import Counter
def structures_count(emails):
    count = Counter()
    for email in emails:
        structure = get_email_structure(email)
        count[structure] += 1
    return count

In [9]:
structures_count(ham_emails).most_common()

[('text/plain', 2408),
 ('multipart(text/plain, application/pgp-signature)', 66),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, video/mng)', 1),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(text/plain, application/x-pkcs7-signature)', 1),
 ('multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain, application/x-java-applet)', 1)]

In [10]:
structures_count(spam_emails).most_common()

[('text/plain', 218),
 ('text/html', 183),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 20),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(text/html, text/plain)', 1),
 ('multipart(multipart(text/html), application/octet-stream, image/jpeg)', 1),
 ('multipart(multipart(text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1)]

In [18]:
email = spam_emails[0]

In [19]:
email.items()

[('Return-Path', '<12a1mailbot1@web.de>'),
 ('Delivered-To', 'zzzz@localhost.spamassassin.taint.org'),
 ('Received',
  'from localhost (localhost [127.0.0.1])\tby phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32\tfor <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)'),
 ('Received',
  'from mail.webnote.net [193.120.211.219]\tby localhost with POP3 (fetchmail-5.9.0)\tfor zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)'),
 ('Received',
  'from dd_it7 ([210.97.77.167])\tby webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623\tfor <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100'),
 ('From', '12a1mailbot1@web.de'),
 ('Received',
  'from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);\t Sat, 24 Aug 2002 09:42:10 +0900'),
 ('To', 'dcek1a1@netsgo.com'),
 ('Subject', 'Life Insurance - Why Pay More?'),
 ('Date', 'Wed, 21 Aug 2002 20:31:57 -1600'),
 ('MIME-Version', '1.0'),
 ('Message-ID', '<0103c104200

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

In [12]:
x = np.array(ham_emails+spam_emails)
y = np.array([0]*len(ham_emails)+[1]*len(spam_emails))
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.S | re.M | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK', text, flags=re.S | re.M | re.I)
    text = re.sub('<.*?>', '', text, flags=re.S | re.M)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.S | re.M)
    return unescape(text)

In [14]:
html_spam_emails = [email for email in X_train[y_train==1]
                    if get_email_structure(email) == "text/html"]
sample_html_spam = html_spam_emails[7]
print(sample_html_spam.get_content().strip()[:1000], "...")

<HTML><HEAD><TITLE></TITLE><META http-equiv="Content-Type" content="text/html; charset=windows-1252"><STYLE>A:link {TEX-DECORATION: none}A:active {TEXT-DECORATION: none}A:visited {TEXT-DECORATION: none}A:hover {COLOR: #0033ff; TEXT-DECORATION: underline}</STYLE><META content="MSHTML 6.00.2713.1100" name="GENERATOR"></HEAD>
<BODY text="#000000" vLink="#0033ff" link="#0033ff" bgColor="#CCCC99"><TABLE borderColor="#660000" cellSpacing="0" cellPadding="0" border="0" width="100%"><TR><TD bgColor="#CCCC99" valign="top" colspan="2" height="27">
<font size="6" face="Arial, Helvetica, sans-serif" color="#660000">
<b>OTC</b></font></TD></TR><TR><TD height="2" bgcolor="#6a694f">
<font size="5" face="Times New Roman, Times, serif" color="#FFFFFF">
<b>&nbsp;Newsletter</b></font></TD><TD height="2" bgcolor="#6a694f"><div align="right"><font color="#FFFFFF">
<b>Discover Tomorrow's Winners&nbsp;</b></font></div></TD></TR><TR><TD height="25" colspan="2" bgcolor="#CCCC99"><table width="100%" border="0" 

In [17]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000], "...")


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Watch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.
Put CBYI on your watch list, acquire a position TODAY.
REASONS TO INVEST IN CBYI
A profitable company and is on track to beat ALL earnings estimates!
One of the FASTEST growing distributors in environmental & safety equipment instruments.
Excellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.
RAPIDLY GROWING INDUSTRY
Industry revenues exceed $900 million, estimates indicate that there could be as much as $25 billi

In [18]:
sample_html_spam.get_payload()

"<HTML><HEAD><TITLE></TITLE><META http-equiv=3D=22Content-Type=22 content=3D=22=\ntext/html; charset=3Dwindows-1252=22><STYLE>A:link =7BTEX-DECORATION: none=7D=\nA:active =7BTEXT-DECORATION: none=7DA:visited =7BTEXT-DECORATION: none=7DA:h=\nover =7BCOLOR: =230033ff; TEXT-DECORATION: underline=7D</STYLE><META conten=\nt=3D=22MSHTML 6.00.2713.1100=22 name=3D=22GENERATOR=22></HEAD>\n<BODY text=3D=22=23000000=22 vLink=3D=22=230033ff=22 link=3D=22=230033ff=22 bgColor=\n=3D=22=23CCCC99=22><TABLE borderColor=3D=22=23660000=22 cellSpacing=3D=220=22 cellP=\nadding=3D=220=22 border=3D=220=22 width=3D=22100%=22><TR><TD bgColor=3D=22=23CCCC99=22=\n valign=3D=22top=22 colspan=3D=222=22 height=3D=2227=22>\n<font size=3D=226=22 face=3D=22Arial, Helvetica, sans-serif=22 color=3D=22=23660=\n000=22>\n<b>OTC</b></font></TD></TR><TR><TD height=3D=222=22 bgcolor=3D=22=236a694f=22>=\n\n<font size=3D=225=22 face=3D=22Times New Roman, Times, serif=22 color=3D=22=23FF=\nFFFF=22>\n<b>&nbsp;Newsletter</b></font>

In [19]:
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ('text/html', 'text/plain'):
            continue
        try:
            content = part.get_content()
        except:
            content = str(part.get_payload())
        if ctype=='text/plain':
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)

In [22]:
print(email_to_text(sample_html_spam)[:100], '...')


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Wat ...


In [23]:
import nltk
import urlextract

In [24]:
stemmer = nltk.PorterStemmer()
url_extractor = urlextract.URLExtract()

In [30]:
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True,
                 replace_urls=True, replace_numbers=True, stemming=True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for email in X:
            text = email_to_text(email) or ''
            if self.lower_case:
                text = text.lower()
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(text)))
                urls.sort(key=lambda url:len(url), reverse=True)
                if urls:
                    for url in urls:
                        text.replace(url, ' URL ')
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            word_counts = Counter(text.split())
            if self.stemming and stemmer is not None:
                stemmed_word_counts = Counter()
                for word, number in word_counts.items():
                    word_stemed = stemmer.stem(word)
                    stemmed_word_counts[word_stemed] += number
                word_counts = stemmed_word_counts
            X_transformed.append(word_counts)
        return np.array(X_transformed)

In [31]:
X_few = X_train[:3]
X_few_wordcounts = EmailToWordCounterTransformer().fit_transform(X_few)
X_few_wordcounts

array([ Counter({'chuck': 1, 'murcko': 1, 'wrote': 1, 'stuff': 1, 'yawn': 1, 'r': 1}),
       Counter({'the': 11, 'of': 9, 'and': 8, 'all': 3, 'christian': 3, 'to': 3, 'by': 3, 'jefferson': 2, 'i': 2, 'have': 2, 'superstit': 2, 'one': 2, 'on': 2, 'been': 2, 'ha': 2, 'half': 2, 'rogueri': 2, 'teach': 2, 'jesu': 2, 'some': 1, 'interest': 1, 'quot': 1, 'http': 1, 'www': 1, 'postfun': 1, 'com': 1, 'pfp': 1, 'worboi': 1, 'html': 1, 'thoma': 1, 'examin': 1, 'known': 1, 'word': 1, 'do': 1, 'not': 1, 'find': 1, 'in': 1, 'our': 1, 'particular': 1, 'redeem': 1, 'featur': 1, 'they': 1, 'are': 1, 'alik': 1, 'found': 1, 'fabl': 1, 'mytholog': 1, 'million': 1, 'innoc': 1, 'men': 1, 'women': 1, 'children': 1, 'sinc': 1, 'introduct': 1, 'burnt': 1, 'tortur': 1, 'fine': 1, 'imprison': 1, 'what': 1, 'effect': 1, 'thi': 1, 'coercion': 1, 'make': 1, 'world': 1, 'fool': 1, 'other': 1, 'hypocrit': 1, 'support': 1, 'error': 1, 'over': 1, 'earth': 1, 'six': 1, 'histor': 1, 'american': 1, 'john': 1, 'e': 1, 'r

In [27]:
X_few

array([<email.message.EmailMessage object at 0x000001E354ECBCF8>,
       <email.message.EmailMessage object at 0x000001E353BA0048>], dtype=object)

In [34]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size=1000):
        self.vocabulary_size = vocabulary_size
    def fit(self, X, y=None):
        total_count = Counter()
        for word_count in X:
            for word, count in word_count.items():
                total_count[word] += min(count, 10)
        most_common = total_count.most_common()[:self.vocabulary_size]
        self.most_common_ = most_common
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(most_common)}
        return self
    def transform(self, X, y=None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape=(len(X), self.vocabulary_size + 1))

In [35]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors

<3x11 sparse matrix of type '<class 'numpy.int32'>'
	with 20 stored elements in Compressed Sparse Row format>

In [36]:
X_few_vectors.toarray()

array([[  6,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [112,  11,   9,   8,   3,   1,   0,   1,   3,   0,   1],
       [ 92,   0,   1,   2,   3,   4,   5,   3,   1,   4,   2]], dtype=int32)

In [37]:
vocab_transformer.vocabulary_

{'all': 8,
 'and': 3,
 'com': 7,
 'http': 5,
 'in': 10,
 'number': 6,
 'of': 2,
 'the': 1,
 'to': 4,
 'yahoo': 9}

In [38]:
from sklearn.pipeline import Pipeline

preprocess_pipeline = Pipeline([
    ("email_to_wordcount", EmailToWordCounterTransformer()),
    ("wordcount_to_vector", WordCounterToVectorTransformer()),
])

X_train_transformed = preprocess_pipeline.fit_transform(X_train)

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
lg = LogisticRegression()
score = cross_val_score(lg, X_train_transformed, y_train, cv=3, verbose=3)
np.mean(score)

[CV]  ................................................................
[CV] ................................. , score=0.981250, total=   0.0s
[CV]  ................................................................
[CV] ................................. , score=0.977500, total=   0.0s
[CV]  ................................................................
[CV] ................................. , score=0.992500, total=   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s finished


0.98375000000000001

## Use sklean.feature_extraction.text

In [59]:
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToTextTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True,
                 replace_urls=True, replace_numbers=True, stemming=True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_transformed = []
        for email in X:
            text = email_to_text(email) or ''
            if self.lower_case:
                text = text.lower()
            if self.replace_urls and url_extractor is not None:
                urls = list(set(url_extractor.find_urls(text)))
                urls = urls.sort(key=lambda url:len(url), reverse=True)
                if urls:
                    for url in urls:
                        text.replace(url, ' URL ')
            if self.replace_numbers:
                text = re.sub(r'\d+(?:\.\d*(?:[eE]\d+))?', 'NUMBER', text)
            if self.remove_punctuation:
                text = re.sub(r'\W+', ' ', text, flags=re.M)
            if self.stemming and stemmer is not None:
                words = text.split()
                stemmed_words = []
                for word in words:
                    stemmed_words.append(stemmer.stem(word))
                text = ' '.join(stemmed_words)
            X_transformed.append(text)
        return np.array(X_transformed)

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

In [75]:
preprocessPipe = Pipeline([('emailtotext', EmailToTextTransformer()),
                 ('conntvector', CountVectorizer(max_features=1000))])
X_train_transformed = preprocessPipe.fit_transform(X_train)

In [76]:
vocab=preprocessPipe.named_steps['conntvector'].vocabulary_

In [77]:
len(vocab)

1000

In [78]:
lg = LogisticRegression()
score = cross_val_score(lg, X_train_transformed, y_train, cv=3, verbose=3)
np.mean(score)

[CV]  ................................................................
[CV] ................................. , score=0.977500, total=   0.0s
[CV]  ................................................................
[CV] ................................. , score=0.981250, total=   0.0s
[CV]  ................................................................
[CV] ................................. , score=0.990000, total=   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s finished


0.98291666666666666